# Visual reprezentation of FlowPics 
#### Author: Jakub Čoček (xcocek00)

Creates visual reprezentation of FlowPics with and without augmentations

In [5]:
# -- IMPORTS --

import os 
os.chdir('/workplace/xcocek00/common/')
from dataloader import create_flowpic_dataloader

os.chdir('/workplace/xcocek00/common/')
from augmentations import (
    augment_iat,
    augment_rtt,
    packet_loss,
)

# set working dir to correct folder
os.chdir('/workplace/xcocek00/flowpics/')

import torchvision.transforms as T
import torchvision.transforms.functional as F
import torch
from PIL import Image, ImageOps
import numpy as np

In [6]:
def flowpic_to_img(tensor, filename, index=0, upscale=16, border=4):
    '''
    Converts FlowPic into upscaled grayscale image.

    Args:
        tensor: tensor of shape [batch_size, 1, H, W]
        index: index of the FlowPic within the batch to save
        filename: path where the output will be stored
        upscale: factor to upscale the original FlowPic resolution
    '''

    # tensor -> np
    arr = tensor[index].squeeze().cpu().numpy()

    # Optional: enhance contrast using log scale
    arr = np.log1p(arr)

    # normalize to 0–255
    arr = arr / arr.max() if arr.max() > 0 else arr
    arr = (1 - arr) * 255  # 0 -> white, max -> black

    img = Image.fromarray(arr.astype(np.uint8), mode='L')

    # resize
    new_size = (img.width * upscale, img.height * upscale)
    img = img.resize(new_size, resample=Image.NEAREST)

    img = ImageOps.expand(img, border=border, fill='black')

    img.save(filename)


In [7]:
def process(fp, upscale, border=4):
    '''
    Process flowpic to visual reprezentation

    Args:
        fp: flowpic tensor
        upscale: the upscale factor
        border: border around final img

    Returns:
        final img
    
    '''
    arr = fp.squeeze().cpu().numpy()
    arr = np.log1p(arr)
    arr = arr / arr.max() if arr.max() > 0 else arr
    arr = (1 - arr) * 255
    
    img = Image.fromarray(arr.astype(np.uint8), mode='L')
    img = img.resize((img.width * upscale, img.height * upscale), resample=Image.NEAREST)
    img = ImageOps.expand(img, border=border, fill='black')
    
    return img

def combine_flowpics_to_img(tensor_orig, tensor_aug, filename, index=0, upscale=16, gap=10):
    '''
    Combines original and augmented FlowPic from into single
    side-by-side upscaled grayscale image.

    Args:
        tensor_orig: original FlowPics
        tensor_aug:  augmented FlowPics
        filename: path where the output will be stored
        index: index of the FlowPic within the batch to save
        upscale: factor to upscale the original FlowPic resolution
        gap: gap between 2 imgs
    '''

    img_orig = process(tensor_orig[index], upscale)
    img_aug  = process(tensor_aug[index], upscale)

    # Calculate combined size with gap
    width = img_orig.width + gap + img_aug.width
    height = max(img_orig.height, img_aug.height)

    combined = Image.new('L', (width, height), color='white')
    combined.paste(img_orig, (0, 0))
    combined.paste(img_aug, (img_orig.width + gap, 0))

    combined.save(filename)


---

FlowPics are created from the Ucdavis-icdm19 dataset, as it achieved the best performance according to my results.

In [9]:
# NO AUGMENTATION

dl_orig = create_flowpic_dataloader(
    dir_path="/workplace/datasets/ucdavis/final-splits/pretraining.csv",
    batch_size=32,
    meta_key="app",    
    time_bins = [i * (15 / 32) for i in range(33)],
    length_bins = [i * (1500 / 32) for i in range(33)],
    bidirectional = False,
)

# one FlowPic only
for fp1_orig, _, _ in dl_orig:
    flowpic_to_img(tensor=fp1_orig, filename="flowpic_no_augmentation.png")
    break

In [10]:
# RTT augmentation

dl = create_flowpic_dataloader(
    dir_path="/workplace/datasets/ucdavis/final-splits/pretraining.csv",
    batch_size=32,
    meta_key="app",    
    time_bins = [i * (15 / 32) for i in range(33)],
    length_bins = [i * (1500 / 32) for i in range(33)],
    bidirectional = False,
    flow_transform_1=augment_rtt,
    flow_transform_2=augment_rtt,
)

# one FlowPic only
for fp1, _, _ in dl:
    flowpic_to_img(tensor=fp1, filename="flowpic_rtt.png")
    combine_flowpics_to_img(tensor_orig=fp1_orig, tensor_aug=fp1, filename="rtt_and_orig.png")
    break

In [11]:
# IAT augmentation

dl = create_flowpic_dataloader(
    dir_path="/workplace/datasets/ucdavis/final-splits/pretraining.csv",
    batch_size=32,
    meta_key="app",    
    time_bins = [i * (15 / 32) for i in range(33)],
    length_bins = [i * (1500 / 32) for i in range(33)],
    bidirectional = False,
    flow_transform_1=augment_iat,
    flow_transform_2=augment_iat,
)

# one FlowPic only
for fp1, _, _ in dl:
    flowpic_to_img(tensor=fp1, filename="flowpic_iat.png")
    combine_flowpics_to_img(tensor_orig=fp1_orig, tensor_aug=fp1, filename="iat_and_orig.png")
    break

In [16]:
# packet loss augmentation

dl = create_flowpic_dataloader(
    dir_path="/workplace/datasets/ucdavis/final-splits/pretraining.csv",
    batch_size=32,
    meta_key="app",    
    time_bins = [i * (15 / 32) for i in range(33)],
    length_bins = [i * (1500 / 32) for i in range(33)],
    bidirectional = False,
    flow_transform_1=packet_loss,
    flow_transform_2=packet_loss,
)

# one FlowPic only
for fp1, _, _ in dl:
    flowpic_to_img(tensor=fp1, filename="flowpic_pkts_loss.png")
    combine_flowpics_to_img(tensor_orig=fp1_orig, tensor_aug=fp1, filename="pktl_and_orig.png")
    break

In [17]:
# FlowPic rotation

dl = create_flowpic_dataloader(
    dir_path="/workplace/datasets/ucdavis/final-splits/pretraining.csv",
    batch_size=32,
    meta_key="app",    
    time_bins = [i * (15 / 32) for i in range(33)],
    length_bins = [i * (1500 / 32) for i in range(33)],
    bidirectional = False,
)

# one FlowPic only
for fp1, _, _ in dl:
    rotate_transform = T.RandomRotation(degrees=(-10,10))
    if fp1.dim() == 4:
        new_fp = torch.stack([rotate_transform(img) for img in fp1])

    flowpic_to_img(tensor=new_fp, filename="flowpic_rotation.png")
    combine_flowpics_to_img(tensor_orig=fp1_orig, tensor_aug=new_fp, filename="rotation_and_orig.png")
    break

/root/miniconda3/lib/python3.12/site-packages/torch/utils/data/datapipes/utils/common.py:145: UserWarning: Local function is not supported by pickle, please use regular python function or functools.partial instead.
  warnings.warn(
